<a href="https://colab.research.google.com/github/sandhyaparna/Python-DataScience-CookBook/blob/master/NLP_in_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
! pip install emoji

     |████████████████████████████████| 133kB 8.6MB/s 


In [3]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

* https://www.rexegg.com/regex-quickstart.html
* https://docs.python.org/3/library/re.html
* http://www.pyregex.com/
* https://www.dataquest.io/blog/regular-expressions-data-scientists/
* All Unicode characters https://en.wikipedia.org/wiki/List_of_Unicode_characters

### Cleaning functions
 

In [ ]:
example_text = ""

tf.strings.lower(example_text)  # converts example_text to lowercase

tf.strings.regex_replace(example_text, '<br />', '') # Replaces line breaks with emptystring in example_text 

# string.punctuation contains !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~   # special characters like these are not removed
# similar to string.punctuation functions https://docs.python.org/3/library/string.html
# re.escape is to escape special characters in above
# % is used to point to string.punctuation
# Any punctuation attached to present seperately in the string is removed
string.punctuation+'®' # to add more characters to string.punctuation
tf.strings.regex_replace(example_text,'[%s]' % re.escape(string.punctuation),'') # Replaces line breaks with emptystring in example_text 

# To convert/encode unicode characters 
example_text.encode("utf-8")
# To decode the format back to unicode character
example_text.dencode("utf-8")

# Extract only text data without labels
raw_train_ds.map(lambda x, y: x)

# To get vocab size of the vectorize layer
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))


### https://www.tensorflow.org/tutorials/keras/text_classification
* In Text Vectorize layer: Each word is given a unique number and encoding of the sentence of the sentence is based on that. Transforms strings into vocab indices
* This Vectorize layer is passed through the keras embedding layer to get embedding vectors for those words. Embedding vector is learnt during training


In [ ]:
### Tensorflow Notebooks
""" Data is extracted from url into folders. Then data is passed through 'tf.keras.preprocessing.text_dataset_from_directory' function to 
prepare data into suitable format """

# Function for a sneak peek into the data </br>
for text_batch, label_batch in raw_train_ds.take(1): 
  for i in range(3): 
    print("Review", text_batch.numpy()[i]) 
    print("Label", label_batch.numpy()[i])  

In [6]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', '')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')
raw_data_sample = 'ത'+'"Pandemonium" -is a horror® movie spoof - !that comes off more stupid than funny.<br /><br />Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
custom_standardization(raw_data_sample)

<tf.Tensor: shape=(), dtype=string, numpy=b'\xe0\xb4\xa4pandemonium is a horror\xc2\xae movie spoof  that comes off more stupid than funnybelieve me when i tell you i love comedies especially comedy spoofs airplane the naked gun trilogy blazing saddles high anxiety and spaceballs are some of my favorite comedies that spoof a particular genre pandemonium is not up there with those films most of the scenes in this movie had me sitting there in stunned silence because the movie wasnt all that funny there are a few laughs in the film but when you watch a comedy you expect to laugh a lot more than a few times and thats all this film has going for it geez scream had more laughs than this film and that was more of a horror film how bizarre is that12 out of four'>

In [ ]:
# ADAPT the vectorize_layer on only Text part of train dat (remove label)
vectorize_layer = TextVectorization(
    standardize=custom_standardization,  # Default is 'lower_and_strip_punctuation'.
    max_tokens=None,  # no cap on number of tokens
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (without labels), then call adapt
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [ ]:
# To retrieve a batch (of 32 reviews and labels) from the dataset: loop is added to the existing code
text_batch, label_batch = next(iter(raw_train_ds))
for i in range(len(text_batch)):
  first_review, first_label = text_batch[i], label_batch[i]
  print("Review", first_review)
  print("Label", raw_train_ds.class_names[first_label])
  print("Vectorized review", vectorize_text(first_review, first_label))

In [ ]:
# To get word associated with a unique number
vectorize_layer.get_vocabulary()[1287]

In [ ]:
text_col = data['feature_text'].values
target_col = data['target'].values

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,'[%s]' % re.escape(string.punctuation),'')

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=None,
    output_mode='int',
    output_sequence_length=None)

vectorize_layer.adapt(text_col) 

model = tf.keras.Sequential([
  vectorize_layer, # Added Vectorize layer here
  Embedding(vocab_size, embedding_dim, name="embedding"),
  tf.keras.layers.LSTM(64),
  tf.keras.layers.Dropout(0.1),
  Dense(1,activation='softplus')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=[tf.keras.metrics.MeanAbsolutePercentageError(), tf.keras.metrics.MeanAbsoluteError()])

In [5]:
print(raw_data_sample)

print(('ത'+raw_data_sample).encode("utf-8"))

print(raw_data_sample.encode("utf-8").decode("utf-8"))


"Pandemonium" -is a horror® movie spoof - !that comes off more stupid than funny.<br /><br />Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn't all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that's all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)
b'\xe0\xb4\xa4"Pandemonium" -is a horror\xc2\xae movie spoof - !that comes off more stupid than funny.<br /><br />Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing S

In [39]:
re.findall(r'[%s]', raw_data_sample)
print(re.findall(r'[%s] % re.escape(string.punctuation)', raw_data_sample))


[]


In [ ]:
 "résumé".encode("utf-8")

 "El Niño".encode("utf-8")


 b"r\xc3\xa9sum\xc3\xa9".decode("utf-8")
'résumé'
.decode("utf-8")
'El Niño'

In [44]:
import emoji

def extract_emojis(s):
  return ''.join(c for c in s if c in emoji.UNICODE_EMOJI)

extract_emojis(raw_data_sample)

''

In [26]:
pattern = re.compile("d")
pattern.search("dog") 

<re.Match object; span=(0, 1), match='d'>